In [1]:
# Import necessary libraries
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import numpy as np

In [2]:
# Load the csv file as dataframe.
df = pd.read_csv('./data/shampoo_products_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product_ID        21 non-null     int64 
 1   Product_Category  21 non-null     object
 2   Brand_Name        21 non-null     object
 3   Product_Name      21 non-null     object
 4   Product_Details   21 non-null     object
 5   Origin            21 non-null     object
 6   Ingredients       21 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.3+ KB


In [4]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"
engine = "PLEASE_ENTER_YOUR_OWNED_AOAI_TEXT_EMBEDDING_MODEL_NAME"

In [5]:
# Perform word embedding for each product name in the dataframe in vector form.
embeddings = [get_embedding(text, engine=engine) for text in df['Product_Name'].values]

In [6]:
# Calculate the cosine similarity between each product name.
similarity_matrix = np.zeros((len(embeddings), len(embeddings)))
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        if i == j:
            similarity_matrix[i,j] = 1.0
        elif i < j:
            similarity_matrix[i,j] = cosine_similarity(embeddings[i], embeddings[j])
        else:
            similarity_matrix[i,j] = similarity_matrix[j,i]

In [7]:
# Save the similarity matrix to a CSV file.
np.savetxt('./data/similarity_matrix_sp_n.csv', similarity_matrix, delimiter=',')

In [8]:
# Reload the similarity matrix from the CSV file.
# similarity_matrix = np.loadtxt('./data/similarity_matrix_sp_n.csv', delimiter=',')

In [9]:
# Create a list of Product_Name.
p_name = pd.Series(df['Product_Name'])
p_name

0                 DOVE BOTANIC BREAKAGE PROTECT SHAMPOO
1               DOVE BOTANIC SPLIT ENDS PROTECT SHAMPOO
2                         DOVE LIGHT MOIST PURE SHAMPOO
3                      DOVE JAPAN AIRY MOISTURE SHAMPOO
4                      DOVE JAPAN RICH MOISTURE SHAMPOO
5                           DOVE SHAMPOO STRAIGHT&SILKY
6             VIDAL SASSOON LIGHT & SOFT SMOOTH SHAMPOO
7                 VIDAL SASSOON MOISTURIZING AD SHAMPOO
8             VIDAL SASSOON TEXTURIZED STRAIGHT SHAMPOO
9           VIDAL SASSOON MOISTURING TRT REPAIR SHAMPOO
10                         REJOICE VOLUME FRESH SHAMPOO
11                                  REJOICE OIL REMOVAL
12                            REJOICE OLIVE OIL SHAMPOO
13                      REJOICE MINT REFRESHING SHAMPOO
14                       REJOICE LEMON BALANCED SHAMPOO
15                            REJOICE 3IN1 CARE SHAMPOO
16        ESSENTIAL NOURISHING BREAKAGE DEFENSE SHAMPOO
17            ESSENTIAL MOISTURIZING FRIZZ FREE 

In [10]:
# Create function for Product Recommendations.
def product_recommendations(Product_Name, cos_sim = similarity_matrix):
    recommended_product = []
    idx = p_name[p_name == Product_Name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_product.append(list(df['Product_Name'])[i])
    return recommended_product

In [13]:
# Perform Product Recommendations. It is based on Product Name to figure out the similarity.
product_recommendations('DOVE BOTANIC BREAKAGE PROTECT SHAMPOO')

['DOVE BOTANIC SPLIT ENDS PROTECT SHAMPOO',
 'ESSENTIAL NOURISHING BREAKAGE DEFENSE SHAMPOO',
 'DOVE LIGHT MOIST PURE SHAMPOO',
 'DOVE SHAMPOO STRAIGHT&SILKY',
 'DOVE JAPAN AIRY MOISTURE SHAMPOO',
 'DOVE JAPAN RICH MOISTURE SHAMPOO',
 'ESSENTIAL MOISTURIZING FRIZZ FREE SHAMPOO',
 'VIDAL SASSOON MOISTURIZING AD SHAMPOO',
 'VIDAL SASSOON TEXTURIZED STRAIGHT SHAMPOO',
 'VIDAL SASSOON LIGHT & SOFT SMOOTH SHAMPOO']

In [14]:
df_product_sim_matrix = pd.DataFrame(similarity_matrix, index = p_name, columns = p_name)
df_product_sim_matrix

Product_Name,DOVE BOTANIC BREAKAGE PROTECT SHAMPOO,DOVE BOTANIC SPLIT ENDS PROTECT SHAMPOO,DOVE LIGHT MOIST PURE SHAMPOO,DOVE JAPAN AIRY MOISTURE SHAMPOO,DOVE JAPAN RICH MOISTURE SHAMPOO,DOVE SHAMPOO STRAIGHT&SILKY,VIDAL SASSOON LIGHT & SOFT SMOOTH SHAMPOO,VIDAL SASSOON MOISTURIZING AD SHAMPOO,VIDAL SASSOON TEXTURIZED STRAIGHT SHAMPOO,VIDAL SASSOON MOISTURING TRT REPAIR SHAMPOO,...,REJOICE OIL REMOVAL,REJOICE OLIVE OIL SHAMPOO,REJOICE MINT REFRESHING SHAMPOO,REJOICE LEMON BALANCED SHAMPOO,REJOICE 3IN1 CARE SHAMPOO,ESSENTIAL NOURISHING BREAKAGE DEFENSE SHAMPOO,ESSENTIAL MOISTURIZING FRIZZ FREE SHAMPOO,Essential Purify Deep Cleansing Care Shampoo,Essential Purify Weightlessly Smooth Care Shampoo,Essential Purify Anti Dandruff Shampoo
Product_Name,,,,,,,,,,,,,,,,,,,,,
DOVE BOTANIC BREAKAGE PROTECT SHAMPOO,1.000000,0.951075,0.920772,0.886769,0.884122,0.888769,0.867286,0.872807,0.868178,0.862930,...,0.776927,0.865434,0.846735,0.850273,0.860685,0.924257,0.878188,0.851140,0.841178,0.846370
DOVE BOTANIC SPLIT ENDS PROTECT SHAMPOO,0.951075,1.000000,0.911165,0.877366,0.877768,0.888563,0.862063,0.865566,0.860589,0.853410,...,0.766043,0.859416,0.839893,0.849201,0.857253,0.884962,0.869227,0.848767,0.833516,0.845442
DOVE LIGHT MOIST PURE SHAMPOO,0.920772,0.911165,1.000000,0.923337,0.922760,0.924643,0.904011,0.893816,0.882420,0.867455,...,0.781793,0.879512,0.871090,0.868557,0.873267,0.872767,0.894854,0.866695,0.866285,0.863525
DOVE JAPAN AIRY MOISTURE SHAMPOO,0.886769,0.877366,0.923337,1.000000,0.973375,0.896943,0.859337,0.878374,0.853405,0.853351,...,0.769998,0.848976,0.851141,0.845329,0.859864,0.846011,0.871093,0.834592,0.831249,0.832639
DOVE JAPAN RICH MOISTURE SHAMPOO,0.884122,0.877768,0.922760,0.973375,1.000000,0.889428,0.853184,0.875891,0.850733,0.851722,...,0.764310,0.849720,0.841929,0.837105,0.853478,0.845813,0.864332,0.830423,0.824960,0.824417
DOVE SHAMPOO STRAIGHT&SILKY,0.888769,0.888563,0.924643,0.896943,0.889428,1.000000,0.876112,0.864327,0.894281,0.843654,...,0.768486,0.855975,0.844240,0.834801,0.850460,0.842561,0.864034,0.825953,0.840335,0.830869
VIDAL SASSOON LIGHT & SOFT SMOOTH SHAMPOO,0.867286,0.862063,0.904011,0.859337,0.853184,0.876112,1.000000,0.959386,0.957166,0.927222,...,0.801224,0.894415,0.881989,0.885769,0.884277,0.881596,0.898368,0.869253,0.880745,0.859773
VIDAL SASSOON MOISTURIZING AD SHAMPOO,0.872807,0.865566,0.893816,0.878374,0.875891,0.864327,0.959386,1.000000,0.946349,0.947564,...,0.806771,0.896637,0.885238,0.879517,0.888808,0.888777,0.917806,0.868749,0.863139,0.865371
VIDAL SASSOON TEXTURIZED STRAIGHT SHAMPOO,0.868178,0.860589,0.882420,0.853405,0.850733,0.894281,0.957166,0.946349,1.000000,0.923800,...,0.790846,0.883214,0.871391,0.868196,0.877006,0.885546,0.897822,0.865653,0.867489,0.861210
